<a href="https://colab.research.google.com/github/kenpurin/Retrosynthesis_Pytorch/blob/main/score_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-06-17 23:07:25--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   212MB/s    in 0.4s    

2021-06-17 23:07:26 (212 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b64

In [1]:
!git clone https://github.com/kenpurin/Retrosynthesis_Pytorch.git

Cloning into 'Retrosynthesis_Pytorch'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 301 (delta 6), reused 2 (delta 2), pack-reused 285
Receiving objects: 100% (301/301), 14.18 MiB | 28.02 MiB/s, done.
Resolving deltas: 100% (169/169), done.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip -q install rdkit-pypi

     |████████████████████████████████| 18.6MB 47.5MB/s 


In [3]:
from rdkit import Chem
import pandas as pd

In [4]:
invalid_smiles_t= True
beam_size=10
#opt_predictions='/content/predictions_USPTO-50K_model_full_version_step_500000_beam10.txt'
opt_targets='/content/Retrosynthesis_Pytorch/USPTO-50K/tgt/tgt-test.txt'
opt_predictions="/content/Retrosynthesis_Pytorch/prediction/predictions_top10_test2000_test16.txt"

In [5]:
def canonicalize_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        return Chem.MolToSmiles(mol, isomericSmiles=True)
    else:
        return ''

In [6]:
def get_rank(row, base, max_rank):
  for i in range(1, max_rank+1):
    #print(row['{}{}'.format(base, i)])
    if row['target'] == row['{}{}'.format(base, i)]:
      return i
  return 0

In [7]:
with open(opt_targets, 'r') as f:
    targets = [''.join(line.strip().split(' ')) for line in f.readlines()]
targets = targets[:]
predictions = [[] for i in range(beam_size)]
test_df = pd.DataFrame(targets)
test_df.columns = ['target']
total = len(test_df)
total

5004

In [8]:
with open(opt_predictions, 'r') as f:
  for i, line in enumerate(f.readlines()):
    predictions[i % beam_size].append(''.join(line.strip().split(' ')))

In [9]:
for i, preds in enumerate(predictions):
    test_df['prediction_{}'.format(i + 1)] = preds
    test_df['canonical_prediction_{}'.format(i + 1)] = test_df['prediction_{}'.format(i + 1)].apply(
    lambda x: canonicalize_smiles(x))  

In [10]:
test_df

,target,prediction_1,canonical_prediction_1,prediction_2,canonical_prediction_2,prediction_3,canonical_prediction_3,prediction_4,canonical_prediction_4,prediction_5,canonical_prediction_5,prediction_6,canonical_prediction_6,prediction_7,canonical_prediction_7,prediction_8,canonical_prediction_8,prediction_9,canonical_prediction_9,prediction_10,canonical_prediction_10
0,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,CC1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,CC1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1,1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,,C1=COCCC1.COC(=O)CC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CC(=O)c1ccc(O)cc1O,NOC(=O)CCC(=O)c1ccc(O)cc1O.OC1CCCCO1,NOC(=O)CCC(=O)c1ccc(O)cc1O.OC1CCCCO1,C1=COCCC1.COC(=O)CCCC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CCCC(=O)c1ccc(O)cc1O,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)c1O,,COC(=O)CCC(=O)c1ccc(O)cc1O.C1=COCCC1,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,C1=CONCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O,C1=CONCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O
1,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ncccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ncccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ncccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ncccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ncccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ncccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ncccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cnccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ccncc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1cnccc1N,COC(=O)c1cccc(C(=O)O)c1.Nc1ccncc1N
2,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)N[C@H]1CC[C@@H](C(=O)O)CC,,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CNOC.CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC,CONC.COC(=O)C1CCC(NC(=O)OC(C)(C)C)CC1,CNOC.COC(=O)C1CCC(NC(=O)OC(C)(C)C)CC1,CONC.CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC
3,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,NCc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,NCc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccc(O)cc1.O=[N+]([O-])c1cccc(Cl)nc1Cl,,Nc1ccc(O)cc1.[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccc(O)cc1.[O-][N+]c1ccc(Cl)nc1Cl,NNc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,NNc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1F,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1F,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)cc1Cl,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)cc1Cl,Nc1cccc(O)c1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1cccc(O)c1.O=[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccc(O)cc1.O[N+]([O-])c1ccc(Cl)nc1Cl,Nc1ccc(O)cc1.[O-][N+](O)c1ccc(Cl)nc1Cl,Nc1ccc(O)c1.O=[N+]([O-])c1ccc(Cl)nc1Cl,
4,[N-]=[N+]=NCC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([...,[N-]=[N+]=NCC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([...,[N-]=[N+]=NCC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([...,N#CC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([N+](=O)[O...,N#CC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([N+](=O)[O...,N#CC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([N+](=O)[O...,N#CC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([N+](=O)[O...,N#CC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([N+](=O)[O...,N#CC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([N+](=O)[O...,N#CC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([N+](=O)[O...,N#CC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([N+](=O)[O...,N#CC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([N+](=O)[O...,N#CC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([N+](=O)[O...,N#CC1=CC[C@@H](c2ccc(Cl)cc2C

In [11]:
test_df['rank'] = test_df.apply(lambda row: get_rank(row, 'canonical_prediction_', beam_size), axis=1)
test_df['rank']

0       1
1       1
2       1
3       1
4       1
       ..
4999    2
5000    1
5001    1
5002    0
5003    5
Name: rank, Length: 5004, dtype: int64

In [12]:
correct = 0
invalid_smiles = 0

for i in range(1, beam_size+1):
    correct += (test_df['rank'] == i).sum()
    print((test_df['rank'] == i).sum(),correct)
    
    invalid_smiles += (test_df['canonical_prediction_{}'.format(i)] == '').sum()
    if invalid_smiles_t:
        print('Top-{}: {:.1f}% || Invalid SMILES {:.2f}%'.format(i, correct/total*100,
                                                                 invalid_smiles/(total*i)*100))
    else:
        print('Top-{}: {:.1f}%'.format(i, correct / total * 100))

2804 2804
Top-1: 56.0% || Invalid SMILES 3.66%
433 3237
Top-2: 64.7% || Invalid SMILES 13.01%
160 3397
Top-3: 67.9% || Invalid SMILES 17.41%
87 3484
Top-4: 69.6% || Invalid SMILES 19.96%
53 3537
Top-5: 70.7% || Invalid SMILES 21.49%
52 3589
Top-6: 71.7% || Invalid SMILES 22.56%
29 3618
Top-7: 72.3% || Invalid SMILES 23.55%
25 3643
Top-8: 72.8% || Invalid SMILES 24.22%
21 3664
Top-9: 73.2% || Invalid SMILES 24.88%
20 3684
Top-10: 73.6% || Invalid SMILES 25.47%


In [ ]:
2747 2747
Top-1: 54.9% || Invalid SMILES 3.04%
503 3250
Top-2: 64.9% || Invalid SMILES 9.34%
184 3434
Top-3: 68.6% || Invalid SMILES 14.16%
109 3543
Top-4: 70.8% || Invalid SMILES 17.78%
65 3608
Top-5: 72.1% || Invalid SMILES 20.49%
46 3654
Top-6: 73.0% || Invalid SMILES 22.79%
42 3696
Top-7: 73.9% || Invalid SMILES 24.90%
30 3726
Top-8: 74.5% || Invalid SMILES 26.76%
18 3744
Top-9: 74.8% || Invalid SMILES 28.54%
10 3754
Top-10: 75.0% || Invalid SMILES 30.84%